In [1]:
import os
import sys
import requests
import json
from langchain_groq import ChatGroq


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
# Append the current and parent directories to the system path
sys.path.append(current_dir)
sys.path.append(parent_dir)

from src.config import settings, logger

os.environ["GROQ_API_KEY"] = settings.GROQ_API_KEY

2025-11-30 22:27:31,613 - src.config - INFO - config.py:49 - <module> - Environment configuration loaded successfully


### Create LLM using Groq

In [3]:
settings.MODEL_NAME = "openai/gpt-oss-20b" # support reasoning
llm = ChatGroq(
    model=settings.MODEL_NAME,
    temperature=settings.TEMPERATURE,
    max_tokens=settings.MAX_TOKENS,
    reasoning_format="parsed",
    timeout=settings.TIMEOUT,
    max_retries=settings.MAX_RETERIES,
)

In [4]:
# Test
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence. "
        "Then give me word to word translations",
    ),
    ("human", "I love programming and Pandas."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

**French translation:**  
> J’aime la programmation et Pandas.

**Word‑by‑word breakdown**

| English | French |
|---------|--------|
| I | Je |
| love | aime |
| programming | la programmation |
| and | et |
| Pandas | Pandas |


### Multimodel Analysis

As of 11.2025, Langchain ChatGroq does not support multimodal inputs. Thus, Groq library is used directly.

In [6]:
from groq import Groq
import base64

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string

image_path = r"..\..\Agentic_Workflow\notebooks\lama.jpg"
base64_image = encode_image_to_base64(image_path)

client = Groq(api_key=settings.GROQ_API_KEY)

completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    }
                }
            ]
        },
        {
            "role": "user",
            "content": "Tell me more about the area."
        }
    ],
    temperature=settings.TEMPERATURE,
    max_completion_tokens=settings.MAX_TOKENS,
)

print(completion.choices[0].message)


ChatCompletionMessage(content='The image features a collection of stickers showcasing llamas, cacti, and hearts on a pink background with white polka dots. The stickers are arranged in two rows, with the top row containing the text "Llama Love You" in black cursive font.\n\n**Sticker Details:**\n\n*   **Llama Stickers:** There are five llama stickers, each with a unique design. They have light-brown fur and are adorned with colorful blankets featuring tassels and hearts.\n*   **Cactus Stickers:** Three cactus stickers are present, each in a different pot. The pots are colored blue, orange, and pink, respectively.\n*   **Heart Stickers:** Two heart stickers are visible, one large pink heart and one smaller pink heart with a white outline.\n\n**Background:**\n\n*   The background of the image is a soft pink color with white polka dots scattered throughout.\n\n**Overall Impression:**\n\n*   The image appears to be a digital illustration or graphic design, likely created for use as sticker

### Tools

In [4]:
from langchain_core.tools import tool

@tool
def get_weather_forcast(loc: str):
    """
    Get the weather forcast of the given day
    :param loc: string loc. Send the input as {'loc': 'loc_value'}
    :return: the weather forcast
    """
    try:
        url = f"https://wttr.in/{loc}?format=j1"
        response = requests.get(url, timeout=20)
        weather_data = response.json()
        area = weather_data["nearest_area"][0]
        area_name = area["areaName"][0]["value"]
        country = area["country"][0]["value"]
        region = area.get("region", [{"value": ""}])[0]["value"]

        location_str = f"{area_name}"
        if region and region != area_name:
            location_str += f", {region}"
        location_str += f", {country}"

        forecast_report = f"📅 **{3}-Day Weather Forecast for {location_str}**\n\n"

        for i, day in enumerate(weather_data["weather"][:3]):
            date = day["date"]
            max_temp_c = day["maxtempC"]
            max_temp_f = day["maxtempF"]
            min_temp_c = day["mintempC"]
            min_temp_f = day["mintempF"]

            # Get the most representative weather description (usually around midday)
            hourly = day["hourly"]
            midday_weather = hourly[len(hourly)//2] if hourly else hourly[0]
            weather_desc = midday_weather["weatherDesc"][0]["value"]

            day_label = "Today" if i == 0 else ("Tomorrow" if i == 1 else date)

            forecast_report += f"**{day_label} ({date}):**\n"
            forecast_report += f"🌡️ High: {max_temp_c}°C ({max_temp_f}°F) | Low: {min_temp_c}°C ({min_temp_f}°F)\n"
            forecast_report += f"☁️ {weather_desc}\n\n"

        # print(forecast_report)
        return forecast_report
    except Exception as e:
        return (f"Error happened during get_weather_forcast call: {e}"
                f"Check the error and if possible, try again.")
        

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

### Create react agent

In [8]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[multiply, get_weather_forcast]
)
response = agent.invoke({"messages": [{"role": "user", "content": "what's 42 x 7?"}]})
response


2025-09-24 23:09:34,218 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-24 23:09:34,421 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content="what's 42 x 7?", additional_kwargs={}, response_metadata={}, id='bbcd78a1-13e8-4b88-9d52-a631daaec984'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the multiply function.', 'tool_calls': [{'id': 'fc_0e3f4edb-6110-479d-adc8-9e3a615beeb1', 'function': {'arguments': '{"a":42,"b":7}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 183, 'total_tokens': 218, 'completion_time': 0.038652656, 'prompt_time': 0.020612352, 'queue_time': 0.319166156, 'total_time': 0.059265008}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--98d6b84e-50d5-4f26-bd64-e3400a012fe8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 42, 'b': 7}, 'id': 'fc_0e3f4edb-6110-479d-adc8-9e3a615beeb1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 183, 'output_to

In [9]:
print(f"**Agent**: {response['messages'][-1].content}")

**Agent**: 42 × 7 = **294**


In [10]:
response = agent.invoke({"messages": [{"role": "user", "content": "How is the weather in Berlin?"}]})
print(response)
print(f"**Agent**: {response['messages'][-1].content}")

2025-09-24 23:09:53,027 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-24 23:09:53,828 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='How is the weather in Berlin?', additional_kwargs={}, response_metadata={}, id='6e87993a-5c69-4a58-a559-7648d194d0bf'), AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to call get_weather_forcast with loc="Berlin".', 'tool_calls': [{'id': 'fc_65344c96-f5cc-488a-838e-05e00b32aa44', 'function': {'arguments': '{"loc":"Berlin"}', 'name': 'get_weather_forcast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 182, 'total_tokens': 221, 'completion_time': 0.038282409, 'prompt_time': 0.010247122, 'queue_time': 0.088031004, 'total_time': 0.048529531}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0d2f656b-8db9-4304-bf9d-279c7e9acdc4-0', tool_calls=[{'name': 'get_weather_forcast', 'args': {'loc': 'Berlin'}, 'id': 'fc_65344c96-f5cc-488a-838e-05e00b32aa44', 'type': 'tool_ca